In [13]:
import numpy as np
import pandas as pd
artist_conv ={}
with open("artist_conversions.csv", 'r') as f:
    for line in f:
        l = line.split(',')
        a_id = l[0]
        artist_name = l[1]
        artist_name = artist_name.strip()
        artist_conv[a_id]  = artist_name

In [14]:
from surprise import Dataset
from surprise import Reader

reader = Reader(sep=',', rating_scale=(0,100))
data = Dataset.load_from_file( 'finalUserData.csv', reader=reader)
data.raw_ratings[:5]


[('1', '1', 1.0, None),
 ('2', '1', 61.0, None),
 ('3', '1', 1.0, None),
 ('4', '1', 1.0, None),
 ('5', '1', 101.0, None)]

In [15]:
from surprise import KNNBaseline
from surprise.model_selection import train_test_split

trainset, testset = train_test_split( data , test_size= .50 , random_state=1)

algo = KNNBaseline( n_factors=10, n_epochs=500, random_state=1 )
algo.fit(trainset)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


In [19]:
from surprise import accuracy
pred = algo.test(testset) 
accuracy.rmse(pred), accuracy.mae(pred)
error = "KNN: " + str(accuracy.rmse(pred))+"\n"
with open("RMSE.txt", "a") as w:
    w.write(error)

RMSE: 28.9102
MAE:  18.7118
RMSE: 28.9102


In [6]:

from collections import defaultdict
def get_top_n(predictions, n=3):
    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n



top_n = get_top_n(pred, n=3)
count = 0
recommendations = []
for uid, user_ratings in top_n.items():
    count+=1
    #print(uid, [iid for (iid, _) in user_ratings])
    ur = [iid for (iid, _) in user_ratings]
    main = str(uid)+","+str(artist_conv[str(ur[0])])+ ","+str(artist_conv[str(ur[1])])+","+str(artist_conv[str(ur[2])])+"\n"
    print(main)
    recommendations.append(main)
print(count)

415,22,426,220

577,204,76,267

277,323,349,216

61,204,158,362

194,204,267,289

198,182,271,227

428,289,216,429

299,31,314,446

539,336,31,252

112,204,476,265

174,289,470,187

53,101,259,332

127,122,227,289

161,31,117,314

510,339,314,161

460,314,346,114

232,140,116,314

23,267,289,470

512,121,398,356

22,314,349,265

575,339,336,57

342,259,336,339

178,212,39,339

117,349,87,64

199,314,346,289

601,117,452,384

67,446,289,217

168,267,314,227

129,31,64,470

360,470,311,479

85,186,349,346

46,117,259,162

222,267,459,182

148,267,289,76

552,267,31,67

455,31,314,204

487,261,314,2

151,31,267,446

167,323,76,267

351,117,252,323

378,314,31,339

582,162,470,64

550,259,349,63

159,31,182,259

418,31,63,117

172,323,314,259

120,204,76,303

280,267,448,31

555,204,267,64

531,289,380,265

245,259,7,162

210,314,182,388

303,31,196,339

548,57,89,259

26,267,284,314

572,31,204,64

525,31,204,216

542,289,227,31

583,336,314,280

154,267,470,314

374,446,48,459

105,289,2

In [7]:
with open("KNN.txt", "w") as w:
    for x in recommendations:
        w.write(x)